## Climbing Shoe Website Parser
## Website: Climb Smart Shop


In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pyshorteners
import pyperclip

Parse for max number of pages to iterate the parser through

In [44]:
Pages = 1

url = "https://climbsmartshop.com/collections/shoes"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
pager_row = soup.find("div", class_="pagination")
pager_row_elements = pager_row.find_all("span", class_="page")
for sale_element in pager_row_elements:
    if sale_element.find("a") != None:
        if int(sale_element.find("a").text) > Pages:
            Pages = int(sale_element.find("a").text)


Parsing through each webpage and converting it into a dataframe

In [54]:
url = "https://climbsmartshop.com/collections/shoes?page=4"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find("div", class_="grid grid--no-gutters grid--uniform")
product_elements = results.find_all("div", class_="grid__item small--one-half medium-up--one-fifth")


for sale_element in product_elements:
    if sale_element.find("s",class_="product-card__regular-price") != None:
        eg = sale_element.find("div", class_="product-card__price").text.strip()[-8:].strip()[1:]
        
        print(eg)

155
164.96
105
105


In [55]:
df = pd.DataFrame(columns=["Shoe","Sale Price","Original Price","Discount","URL","Shipping Cost"])

for i in range(1,Pages+1):
    url = "https://climbsmartshop.com/collections/shoes?page={}".format(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find("div", class_="grid grid--no-gutters grid--uniform")
    product_elements = results.find_all("div", class_="grid__item small--one-half medium-up--one-fifth")

    for sale_element in product_elements:
        if sale_element.find("s",class_="product-card__regular-price") != None:
            new_row = {
                "Shoe" : sale_element.find("div", class_="product-card__info").find("div", class_="product-card__brand").text.strip() 
                + " "
                + sale_element.find("div", class_="product-card__info").find("div", class_="product-card__name").text.strip(),
                "Sale Price" : float(sale_element.find("div", class_="product-card__price").text.strip()[-8:].strip()[1:]),
                "Original Price" : float(sale_element.find("div", class_="product-card__price").find("s",class_="product-card__regular-price").text.strip()[1:]),
                "Discount" : str(round((float(sale_element.find("div", class_="product-card__price").find("s",class_="product-card__regular-price").text.strip()[1:])-float(sale_element.find("div", class_="product-card__price").text.strip()[-8:].strip()[1:]))/float(sale_element.find("div", class_="product-card__price").find("s",class_="product-card__regular-price").text.strip()[1:]),2)*100)+"%",
                "URL" : "climbsmartshop.com" + sale_element.find("a")["href"],
                "Shipping Cost" : 15
            }
            new_row_df = pd.DataFrame([new_row])
            df = pd.concat([df, new_row_df],ignore_index=True)


type_tiny = pyshorteners.Shortener() 
df["URL"] = df["URL"].apply(lambda x: type_tiny.tinyurl.short(x))
df["Post-Tax Price"] = (df["Sale Price"]+df["Shipping Cost"])*1.13

Output Table

In [56]:
#Output the table with values sorted
df.sort_values(by="Discount",ascending=False)

,Shoe,Sale Price,Original Price,Discount,URL,Shipping Cost,Post-Tax Price
18,La Sportiva Solution (Women's) 2017,105.00,210.00,50.0%,https://tinyurl.com/26zzfheg,15,135.6
11,Scarpa Force V,85.00,170.00,50.0%,https://tinyurl.com/26uwo6kz,15,113.0
17,La Sportiva Solution 2017,105.00,210.00,50.0%,https://tinyurl.com/2xogkcn9,15,135.6
0,Evolv Defy (black/sulphur),70.00,135.00,48.0%,https://tinyurl.com/2cnawvyr,15,96.05
13,SoiLL The Runner LV,80.00,142.00,44.0%,https://tinyurl.com/2bay4yuf,15,107.35
14,SoiLL The New Zero,135.00,210.00,36.0%,https://tinyurl.com/2dqbbdcz,15,169.5
15,La Sportiva Genius,155.00,235.00,34.0%,https://tinyurl.com/25h8nfdn,15,192.1
3,Evolv Venga Rock Shoes - Lime Green- Children,40.00,60.00,33.0%,https://tinyurl.com/2xjqxxsv,15,62.15
6,Mad Rock Mad Rock Drone LV Black (Low Volume),124.60,178.00,30.0%,https://tinyurl.com/2a5a669q,15,157.748
9,Evolv Rave,108.50,155.00,30.0%,https://tinyurl.com/24xvqkov,15,139.555
